# BERT Chatbot POC

To build a chatbot, i will devide the process in 4 diffrent step 
- Importing and reading the PDF files 
- Importing COQA data set 
- Building the chatbot using BERT 
- Merging the model and the PDF retrived data 


In [2]:
#Importing the required liberaries
import numpy as np 
import pandas as pd
import PyPDF2
import glob
#import nltk 
import re
import random 
import string 
#import os
import torch
from transformers import BertTokenizerFast,BertForQuestionAnswering
pd.set_option('display.max_colwidth', 100)

# Importing and reading PDF files 


To import and read PDF files PyPDF2 liberary is used to read the PDF pages and extract text from them. 
To retrive the PDF files stored in a folder Glob is used to return all file paths to match a specific pattern.

In [3]:
pdf_dir ="C:/Users/Arush/OneDrive/Desktop/sample"
pdf_files = glob.glob("%s/*.pdf" % pdf_dir , recursive = True)

text = " "
# creating a pdf file object 
for pdf_file in pdf_files:
    pdfFileObj = open(pdf_file, 'rb') 
    # creating a pdf reader object 
 
    pdfReader = PyPDF2.PdfFileReader(pdfFileObj, strict= False) 
    # printing number of pages in pdf file 
    print(pdfReader.numPages) 
    Page = pdfReader.getNumPages()
    
   
    for i in range(0,Page):
        # creating a page object 
        pageObj = pdfReader.getPage(i) 
        # extracting text from page 
        text += pageObj.extractText()
    pdfFileObj.close()     

1


In [4]:
#using Regular Expression for cleaning the text 

PDF_text = re.sub(r'\[[0-9]*\]',' ', text)
PDF_text = re.sub(r'\s+',' ', text)

PDF_text

' I would like to get your all thoughts on the bond yield increase this week. I am not worried about the market downturn but the sudden increase in yields. On 2/16 the 10 year bonds yields increased by almost 9 percent and on 2/19 the yield increased by almost 5 percent. Key Points from the CNBC Article: to market panic after the Federal Reserve announced that it would begin tapering its quantitative easing program.** * **Major central banks around the world have cut interest rates to historic lows and launched unprecedented quantities of asset purchases in a bid to shore up the economy throughout the pandemic.** * **However, the recent r ise in yields suggests that some investors are starting to anticipate a tightening of policy sooner than anticipated to accommodate a potential rise in inflation.** The recent rise in bond yields and U.S. inflation expectations has some investors wary tha horizon. The benchmark U.S. 10 - year Treasury note climbed above 1.3% for the first time since F

# Importing COQA data set 

CoQA (conversational question answering dataset) is large scaled data used for building conversational question answering system. It contains 127,000+ questions with answers collected from 8000+ conversations

In [5]:
#Loading Data 
coqa = pd.read_json('http://downloads.cs.stanford.edu/nlp/data/coqa/coqa-train-v1.0.json')

In [6]:
coqa

,version,data
0,1,"{'source': 'wikipedia', 'id': '3zotghdk5ibi9cex97fepx7jetpso7', 'filename': 'Vatican_Library.txt..."
1,1,"{'source': 'cnn', 'id': '3wj1oxy92agboo5nlq4r7bndc3t8a8', 'filename': 'cnn_fe05c61a7e48461f7883c..."
2,1,"{'source': 'gutenberg', 'id': '3bdcf01ogxu7zdn9vlrbf2rqzwplyf', 'filename': 'data/gutenberg/txt/..."
3,1,"{'source': 'cnn', 'id': '3ewijtffvo7wwchw6rtyaf7mfwte0p', 'filename': 'cnn_0c518067e0df811501e46..."
4,1,"{'source': 'gutenberg', 'id': '3urfvvm165iantk80llvkwwbjs7uzh', 'filename': 'data/gutenberg/txt/..."
...,...,...
7194,1,"{'source': 'gutenberg', 'id': '34j10vatjfyw0aohj8d4a0wwku3qif', 'filename': 'data/gutenberg/txt/..."
7195,1,"{'source': 'cnn', 'id': '3vj40nv2qinjocrcy7k4z235g6aotn', 'filename': 'cnn_ef5596ece4a9de118290e..."
7196,1,"{'source': 'race', 'id': '3rjsc4xj10uw0to3vq0v6l191rz05q', 'filename': 'middle8160.txt', 'story'..."
7197,1,"{'source': 'wikipedia', 'id': '3gs6s824sqxty8vusxp27xazutmnw2', 'filename': 'Frankfurt.txt', 'st..."


## Data Cleaning 

In [7]:
#Deleting version column 
del coqa["version"]

The Bert model for Question answering trains the model with the data in the form of question context pair. So changing the data in the required format and linking the question and answer to the linked story.

In [8]:

#required columns in a dataframe 
cols = ['text','question','answer']

comp_list =[]
for index, row in coqa.iterrows():
    for i in range(len(row['data']['questions'])):
        temp_list = []
        temp_list.append(row['data']['story'])
        temp_list.append(row['data']['questions'][i]['input_text'])
        temp_list.append(row['data']['answers'][i]['input_text'])
        comp_list.append(temp_list)
        
new_df = pd.DataFrame(comp_list,columns = cols)




In [9]:
# saving the dataframe in to csv 

new_df.to_csv('coqa_data.csv',index = False)

## Loading the data 

In [10]:
data = pd.read_csv('coqa_data.csv')
data

,text,question,answer
0,"The Vatican Apostolic Library (), more commonly called the Vatican Library or simply the Vat, is...",When was the Vat formally opened?,It was formally established in 1475
1,"The Vatican Apostolic Library (), more commonly called the Vatican Library or simply the Vat, is...",what is the library for?,research
2,"The Vatican Apostolic Library (), more commonly called the Vatican Library or simply the Vat, is...",for what subjects?,"history, and law"
3,"The Vatican Apostolic Library (), more commonly called the Vatican Library or simply the Vat, is...",and?,"philosophy, science and theology"
4,"The Vatican Apostolic Library (), more commonly called the Vatican Library or simply the Vat, is...",what was started in 2014?,a project
...,...,...,...
108642,(CNN) -- Cristiano Ronaldo provided the perfect riposte to FIFA president Sepp Blatter after sco...,Who was a sub?,Xabi Alonso
108643,(CNN) -- Cristiano Ronaldo provided the perfect riposte to FIFA president Sepp Blatter after sco...,Was it his first game this year?,Yes
108644,(CNN) -- Cristiano Ronaldo provided the perfect riposte to FIFA president Sepp Blatter after sco...,What position did the team reach?,third
108645,(CNN) -- Cristiano Ronaldo provided the perfect riposte to FIFA president Sepp Blatter after sco...,Who was ahead of them?,Barca.


# Building the Chatbot 

In [11]:
tokenizer = BertTokenizerFast.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

In [12]:
#Randomly selecting a question number from COQA dataset to train the model

random_num = np.random.randint(8,len(data))

question = data['question'][random_num]
text = data['text'][random_num]

In [13]:
print(question, '\n' ,text)

What did Betty do as John told her about it? 
 CHAPTER XXIX 

A REPRESENTATIVE GATHERING 

When John returned to the office, he found that his absence had been causing Betty an anxious hour's waiting. She had been informed by Pugsy that he had gone out in the company of Mr. Parker, and she felt uneasy. She turned white at his story of the ride, but he minimized the dangers. 

"I don't think he ever meant to shoot. I think he was going to shut me up somewhere out there, and keep me till I promised to be good." 

"Do you think my stepfather told him to do it?" 

"I doubt it. I fancy Parker is a man who acts a good deal on his own inspirations. But we'll ask him, when he calls to-day." 

"Is he going to call?" 

"I have an idea he will," said John. "I sent him a note just now, asking if he could manage a visit." 

It was unfortunate, in the light of subsequent events, that Mr. Jarvis should have seen fit to bring with him to the office that afternoon two of his collection of cats, and tha

In [14]:
#tokenzinging text and question as a pair for the model to read in the format of [cls]question[sep]context[sep]
input_ids = tokenizer.encode_plus(question,text)
print(input_ids.keys())
print("The input has a total of {} tokens.".format(len(input_ids['input_ids'])))

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])
The input has a total of 357 tokens.


In the output above tokenization created 3 tensors ['input_ids', 'token_type_ids', 'attention_mask']. we only need input_ids and token_type_ids 

In [15]:
tokens = tokenizer.convert_ids_to_tokens(input_ids['input_ids'])
#print(' ,'.join(tokens))

    
    # For each token and its id...
for token, id in zip(tokens, input_ids ['input_ids']):
    
    # If this is the [SEP] token, add some space around it to make it stand out.
    if id == tokenizer.sep_token_id:
        print('')
    
    # Print the token string and its ID in two columns.
    print('{:<12} {:>6,}'.format(token, id))

    if id == tokenizer.sep_token_id:
        print('')
    

[CLS]           101
what          2,054
did           2,106
betty         9,306
do            2,079
as            2,004
john          2,198
told          2,409
her           2,014
about         2,055
it            2,009
?             1,029

[SEP]           102

chapter       3,127
xx           22,038
##ix          7,646
a             1,037
representative  4,387
gathering     7,215
when          2,043
john          2,198
returned      2,513
to            2,000
the           1,996
office        2,436
,             1,010
he            2,002
found         2,179
that          2,008
his           2,010
absence       6,438
had           2,018
been          2,042
causing       4,786
betty         9,306
an            2,019
anxious      11,480
hour          3,178
'             1,005
s             1,055
waiting       3,403
.             1,012
she           2,016
had           2,018
been          2,042
informed      6,727
by            2,011
pu           16,405
##gs          5,620
##y           2,

In [16]:
#token input_ids to represent the input
#token segment_ids to differentiate our segments - text and question
output = model(torch.tensor([input_ids['input_ids']]),  token_type_ids=torch.tensor([input_ids['token_type_ids']]))
print(output.start_logits, output.end_logits)

tensor([[-0.7795, -6.5012, -7.9509, -8.1476, -7.4495, -8.2879, -7.9080, -7.3919,
         -7.9965, -8.0105, -8.3358, -9.6904, -0.7795, -6.0707, -8.0976, -7.7681,
         -5.7102, -5.0340, -6.6264, -4.2509, -4.0900, -4.0228, -6.3500, -6.1168,
         -5.5615, -7.0696, -4.8340, -5.5944, -6.7128, -1.9254, -3.0223, -7.5082,
         -6.9446, -4.8248, -1.6756, -2.9204, -2.7060, -4.9209, -7.1110, -6.8519,
         -2.9864, -0.7796,  0.8783, -4.1965, -4.3146, -2.0154, -4.7170, -2.0687,
         -6.3176, -6.9604, -3.2934, -0.8270, -2.6903,  1.4765, -1.3313, -3.9548,
         -3.6135, -3.1975, -6.6226, -1.3334, -6.7157, -2.7310, -6.4445, -4.2075,
          0.7457, -0.6697, -2.0554, -3.7968,  4.0790,  3.5689, -0.1246, -3.7520,
         -0.6827, -0.8229, -4.7659, -0.0798, -0.3039, -5.2038, -1.4983,  0.6813,
         -0.4787, -4.0973, -2.8769, -1.2445, -4.5417, -0.7143, -1.5973, -3.5011,
         -6.4219, -4.9608, -4.5145, -2.8850, -6.1777, -4.9875, -5.6112, -2.8412,
         -6.3343, -2.9686, -

 The final embedding is fed to start token classifier layer. Start token classifier has single set of weights which it applies 
 to every word. After taking the dot product between the output embeddings and the ‘start’ weights, model apply the softmax 
 activation to produce a probability distribution over all of the words. Whichever word has the highest probability of 
 being the start token is the one that model pick, for the end token model have a separate weight vector.  

 Model predicted the best answer by selecting the start and end tokens (start+end combination) with the largest logits.
 An answer wherein the end token falls before the start token should be excluded
 Candidate answers wherein the start or end tokens are associated with question tokens are also excluded

In [17]:
# looking at the most probable start and end words 
answer_start = torch.argmax(output.start_logits) # the token with highest probability of being start of the answer
answer_end = torch.argmax(output.end_logits) # the token with highest probability of being end of the answer

print(answer_start, answer_end)

tensor(68) tensor(70)


In [19]:
# providing answers only if the end token is after the start token 

if answer_end >= answer_start:
    answer =  " ".join(tokens[answer_start:answer_end+1])
else:
    print("Sorry! I need more information to answer this question. Can you ask another question?")
    
#print("Text:\n{}".format(text.capitalize()))
print("\nQuestion:\n{}".format(question.capitalize()))
print("\nAnswer:\n{}.".format(answer.capitalize()))


Question:
What did betty do as john told her about it?

Answer:
She turned white.


BERT uses wordpiece tokenization and vocabulary size of 30,000. In BERT, rare words get broken down into subwords/pieces.
Wordpiece tokenization uses ## to delimit tokens that have been split, for example "Playing" is split in to "Play' and "##ing"

In [20]:
# combining the broken words 

# Start with the first token.
answer = tokens[answer_start]
# Select the remaining answer tokens and join them with whitespace.
for i in range(answer_start+1, answer_end+1):
    # If it's a subword token, then recombine it with the previous token.
    if tokens[i][0:2] == '##':
        answer += tokens[i][2:]
    # Otherwise, add a space then the token.
    else:
        answer += " " +tokens[i]

In [21]:
# converting the process in to a function 


def question_answer(question, text):
    
    #tokenize question and text in ids as a pair
    input_ids = tokenizer.encode(question, text)
    
    #string version of tokenized ids
    tokens = tokenizer.convert_ids_to_tokens(input_ids)
    
    # Search the input_ids for the first instance of the `[SEP]` token.
    sep_idx = input_ids.index(tokenizer.sep_token_id)

    #number of tokens in segment A includes the [SEP] token istelf. - question
    num_seg_a = sep_idx+1

    #number of tokens in segment B - text
    num_seg_b = len(input_ids) - num_seg_a
    
    # constructing the list of 0s and 1s
    segment_ids = [0]*num_seg_a + [1]*num_seg_b
    
    # There should be a segment_id for every input token.
    assert len(segment_ids) == len(input_ids)
    
    #model output using input_ids and segment_ids
    output = model(torch.tensor([input_ids]), token_type_ids=torch.tensor([segment_ids]))
    
    #reconstructing the answer
    answer_start = torch.argmax(output.start_logits)
    answer_end = torch.argmax(output.end_logits)

    if answer_end >= answer_start:
        answer = tokens[answer_start]
        for i in range(answer_start+1, answer_end+1):
            if tokens[i][0:2] == "##":
                answer += tokens[i][2:]
            else:
                answer += " " + tokens[i]
                
    if answer.startswith("[CLS]"):
        answer = "Sorry! I need more information to answer this question. Can you ask another question?."
    print("\nAnswer:\n{}".format(answer.capitalize()))  

In [22]:
text = PDF_text
question = ' on 2/19 the yield increased by?'
question_answer(question,text)


Answer:
5 percent


# Merging the model and the PDF retrived data

In [27]:
print('Hello! i am Max. How i can help you')
text = PDF_text
question = input()

while True:
    question_answer(question, text)
    
    flag = True
    flag_N = False
    
    while flag:
        response = input("\nDo you want to ask another question based on this text (Y/N)? ")
        if response[0] == "Y":
            question = input("\nPlease enter your question: \n")
            flag = False
        elif response[0] == "N":
            print("\nBye!")
            flag = False
            flag_N = True
            
    if flag_N == True:
        break

Hello! i am Max. How i can help you
 on 2/19 the yield increased by what percentage ?

Answer:
5

Do you want to ask another question based on this text (Y/N)? Y

Please enter your question: 
on 2/19 the yield increased by?

Answer:
5 percent

Do you want to ask another question based on this text (Y/N)? Y

Please enter your question: 
who is chief investment officer at Beaufort Investment?

Answer:
Balkham

Do you want to ask another question based on this text (Y/N)? N

Bye!
